### Importing dependencies

In [2]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset


## Step 1 Processing a Dataset

#### Downloading the dataset

In [8]:
# Download data from open datasets\n",
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)



file_sizes: 100%|███████████████████████████| 10.8M/10.8M [00:17<00:00, 624kB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


#### Divide data into training and testing set

In [5]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')


#### Print the names of the data columns contained in the dataset for dataset pre-processing.

In [6]:
print(train_dataset.get_col_names())


['image', 'label']


#### transform the image data and the label using map

In [7]:
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]

    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset


#### pack the processed dataset into a batch of size 64.

In [9]:
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)


#### create_tuple_ iterator to iterate over the dataset, printing the shape and dtype for image and label.

In [10]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break


Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


create_dict iterator to iterate over the dataset, printing the shape and dtype for image and label.

In [13]:
for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break


Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


## Step 2 Building Network

In [14]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)


Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


## Step 3: Training Model

In [15]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

# 1. Define forward function
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

# 2. Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

# 3. Define function of one-step training
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


#### define test functions that are used to evaluate the performance of the model.

In [16]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [17]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset)
    test(model, test_dataset, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.316030  [  0/938]
loss: 1.632870  [100/938]
loss: 0.895270  [200/938]
loss: 0.572116  [300/938]
loss: 0.750103  [400/938]
loss: 0.434709  [500/938]
loss: 0.301600  [600/938]
loss: 0.328395  [700/938]
loss: 0.502947  [800/938]
loss: 0.445527  [900/938]
Test: 
 Accuracy: 90.6%, Avg loss: 0.321591 

Epoch 2
-------------------------------
loss: 0.419800  [  0/938]
loss: 0.204912  [100/938]
loss: 0.276902  [200/938]
loss: 0.372570  [300/938]
loss: 0.355444  [400/938]
loss: 0.232125  [500/938]
loss: 0.138512  [600/938]
loss: 0.298223  [700/938]
loss: 0.250386  [800/938]
loss: 0.209825  [900/938]
Test: 
 Accuracy: 92.8%, Avg loss: 0.247309 

Epoch 3
-------------------------------
loss: 0.243151  [  0/938]
loss: 0.144813  [100/938]
loss: 0.182992  [200/938]
loss: 0.284947  [300/938]
loss: 0.159624  [400/938]
loss: 0.251746  [500/938]
loss: 0.341027  [600/938]
loss: 0.090185  [700/938]
loss: 0.183767  [800/938]
loss: 0.394843  [900/938]
Test: 
 

## Step 4 Saving a Model - saving the model parameters

In [18]:
# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")


Saved Model to model.ckpt


## Step 5 Loading a Model

In [19]:
# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
# - param_not_load is the unloaded parametr list. If it is empty allthe parameters have been loaded.
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)


[]


#### Using the loaded model directly for predictive inference.

In [20]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break


Predicted: "[5 2 1 2 9 5 8 3 1 5]", Actual: "[5 2 1 2 9 5 3 2 1 5]"
